In [2]:
import gffpandas.gffpandas as gffpd
import pandas as pd

PATH="/lustre/fs5/jarv_lab/scratch/adenisova/Inno_2025/genome_data/ncbi_dataset/data"

single_orthogroups = pd.read_csv(
    f'{PATH}/longest_isoforms_proteomes_clean/OrthoFinder/Results_Dec02/Orthogroups/Orthogroups_SingleCopyOrthologues.txt', 
    names=["Orthogroup"]
)

orthogroups = pd.read_csv(
    f'{PATH}/longest_isoforms_proteomes_clean/OrthoFinder/Results_Dec02/Orthogroups/Orthogroups.txt', 
    sep=":", 
    names=["Orthogroup", "Genes"]
)

single_orthogroups_df = single_orthogroups.merge(orthogroups)

single_orthogroups_df["rna"] = single_orthogroups_df["Genes"].apply(
    lambda x: [i.split("GCF_036417665.1")[1][1:] for i in x.split() if i.startswith("GCF_036417665.1")][0]
)
single_orthogroups_df["rna"]

df_gff = gffpd.read_gff3(
    f'{PATH}/GCF_036417665.1/genomic.gff'
).df

mRNA_df = df_gff[df_gff["type"] == 'mRNA']
mRNA_df["rna"] = mRNA_df["attributes"].apply(
    lambda x: x.split("ID=")[1].split(";")[0]
)

# mRNA_df["rna"].merge(single_orthogroups_df)

mRNA_df["gene_name"] = mRNA_df["attributes"].apply(
    lambda x: x.split("Parent=")[1].split(";")[0].split("-")[1]
)
name_orthogroup = mRNA_df[["gene_name", "rna"]].merge(single_orthogroups_df, on="rna")[
    ["gene_name", "Orthogroup"]
]

/tmp/ipykernel_2461518/1008991505.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mRNA_df["rna"] = mRNA_df["attributes"].apply(
/tmp/ipykernel_2461518/1008991505.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mRNA_df["gene_name"] = mRNA_df["attributes"].apply(


In [3]:
selection_df = pd.read_csv(
    f'{PATH}/busted_e_summary_per_gene_filter.csv' 
).rename(columns={"gene": "Orthogroup"})

df_merged = selection_df.merge(name_orthogroup)

df_merged.to_csv("busted_per_gene_with_names.csv", index=False)

In [4]:
df_busted = pd.read_csv(f"{PATH}/busted_e_vs_busted_table.csv")
df_busted.head()

df_busted['Orthogroup'] = df_busted['gene']

In [5]:
df_merged_b = df_merged.merge(df_busted)
df_merged_b_selected = df_merged_b[df_merged_b["fdr_e"] < 0.05][[
    "Orthogroup",	"n_branches",	"n_sig", "sig_branches", "gene_name"
]]

f = open(f"{PATH}/OG_selected_absrel.txt", "w")
f.write(
    '\n'.join(df_merged_b_selected[df_merged_b_selected["n_sig"] >= 3]["Orthogroup"].tolist())
)
f.close()


df_merged_b_selected.to_csv(f"{PATH}/busted_per_gene_with_names_and_branches_selected.csv", index=False)

In [10]:
df_merged_b.to_csv(f"{PATH}/busted_per_gene_with_names_and_branches.csv", index=False)

In [6]:
df_merged_b_selected.head(20)

,Orthogroup,n_branches,n_sig,sig_branches,gene_name
1,OG0008541,6,4,"['Buteo_buteo', 'Falco_peregrinus', 'Hirundo_r...",MYO18A
2,OG0009287,6,3,"['Buteo_buteo', 'Hirundo_rustica', 'Passer_dom...",RASSF5
3,OG0003346,6,3,"['Falco_peregrinus', 'Hirundo_rustica', 'Passe...",UBE3D
4,OG0005100,6,3,"['Buteo_buteo', 'Falco_peregrinus', 'Passer_do...",TEX36
6,OG0008296,6,3,"['Buteo_buteo', 'Falco_peregrinus', 'Passer_do...",UNK
7,OG0003196,6,3,"['Buteo_buteo', 'Falco_peregrinus', 'Passer_do...",GINM1
9,OG0009537,6,3,"['Buteo_buteo', 'Hirundo_rustica', 'Passer_dom...",NPPA
11,OG0008365,6,3,"['Falco_peregrinus', 'Hirundo_rustica', 'Passe...",RGS9
12,OG0006126,6,3,"['Buteo_buteo', 'Falco_peregrinus', 'Hirundo_r...",RBFOX1
15,OG0009794,6,3,"['Buteo_buteo', 'Hirundo_rustica', 'Passer_dom...",C28H8orf58


In [17]:
df_merged

,Orthogroup,n_branches,n_sig,sig_branches,fraction_sig,gene_name
0,OG0002065,6,4,"['Buteo_buteo', 'Falco_peregrinus', 'Hirundo_r...",0.666667,LOC135294711
1,OG0008541,6,4,"['Buteo_buteo', 'Falco_peregrinus', 'Hirundo_r...",0.666667,MYO18A
2,OG0009287,6,3,"['Buteo_buteo', 'Hirundo_rustica', 'Passer_dom...",0.500000,RASSF5
3,OG0003346,6,3,"['Falco_peregrinus', 'Hirundo_rustica', 'Passe...",0.500000,UBE3D
4,OG0005100,6,3,"['Buteo_buteo', 'Falco_peregrinus', 'Passer_do...",0.500000,TEX36
...,...,...,...,...,...,...
365,OG0006313,6,0,[],0.000000,LOC135305224
366,OG0009616,6,0,[],0.000000,LOC135284566
367,OG0001945,6,0,[],0.000000,NKIRAS1
368,OG0004273,6,0,[],0.000000,FAR1


In [94]:
import gffpandas.gffpandas as gffpd
import pandas as pd

PATH="/lustre/fs5/jarv_lab/scratch/adenisova/Inno_2025/genome_data/ncbi_dataset/data"

df = pd.read_csv(f'{PATH}/absrel_summary_per_gene_10.csv')

single_orthogroups = pd.read_csv(
    f'{PATH}/longest_isoforms_proteomes_clean/OrthoFinder/Results_Dec02/Orthogroups/Orthogroups_SingleCopyOrthologues.txt', 
    names=["Orthogroup"]
)

orthogroups = pd.read_csv(
    f'{PATH}/longest_isoforms_proteomes_clean/OrthoFinder/Results_Dec02/Orthogroups/Orthogroups.txt', 
    sep=":", 
    names=["Orthogroup", "Genes"]
)

single_orthogroups_df = single_orthogroups.merge(orthogroups)

single_orthogroups_df["rna"] = single_orthogroups_df["Genes"].apply(
    lambda x: [i.split("GCF_036417665.1")[1][1:] for i in x.split() if i.startswith("GCF_036417665.1")][0]
)
single_orthogroups_df["rna"]

df_gff = gffpd.read_gff3(
    f'{PATH}/GCF_036417665.1/genomic.gff'
).df

mRNA_df = df_gff[df_gff["type"] == 'mRNA']
mRNA_df["rna"] = mRNA_df["attributes"].apply(
    lambda x: x.split("ID=")[1].split(";")[0]
)

# mRNA_df["rna"].merge(single_orthogroups_df)

mRNA_df["gene_name"] = mRNA_df["attributes"].apply(
    lambda x: x.split("Parent=")[1].split(";")[0].split("-")[1]
)
name_orthogroup = mRNA_df[["gene_name", "rna"]].merge(single_orthogroups_df, on="rna")[
    ["gene_name", "Orthogroup"]
]

/tmp/ipykernel_2398033/4209609022.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mRNA_df["rna"] = mRNA_df["attributes"].apply(
/tmp/ipykernel_2398033/4209609022.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mRNA_df["gene_name"] = mRNA_df["attributes"].apply(


In [95]:
selection_df = pd.read_csv(
    f'{PATH}/absrel_summary_per_gene.csv'
).rename(columns={"gene": "Orthogroup"})

selection_df.merge(name_orthogroup).to_csv("absrel_per_gene_12_with_names.csv", index=False)

In [76]:
name_orthogroup

,gene_name,Orthogroup
0,CHPF2,OG0001757
1,ABCF2,OG0001758
2,LOC135286772,OG0001759
3,ASB10,OG0001760
4,AGAP3,OG0001761
...,...,...
8950,IL23A,OG0009697
8951,THUMPD2,OG0010056
8952,MAPK15,OG0010057
8953,CFAP298,OG0001756
